# 4 points in 3D with Delaunay

In [1]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay

In [3]:
lbox=10

n_vertices = 4
max_module = 2.0

vertices = np.random.uniform(-lbox+max_module, lbox-max_module, (n_vertices, 3))
v_vertices = np.random.uniform(-1, 1, (n_vertices, 3))
v_vertices = max_module*v_vertices/np.linalg.norm(np.ravel(v_vertices))

In [4]:
vertices[0,:]=[-5,5,0]
vertices[1,:]=[-5,-5,0]
vertices[2,:]=[7,0,0]
vertices[3,:]=[0,0,7]

In [9]:
def barycentric_coords3(vertices, point):
    T = (np.array(vertices[:-1])-vertices[-1]).T
    v = np.dot(la.inv(T), np.array(point)-vertices[-1])
    v.resize(len(vertices))
    v[-1] = 1-v.sum()
    return v

In [30]:
def barycentric_coords3_delaunay(delaunay, point):
    simplex = delaunay.find_simplex(point)
    ndim = delaunay.transform.shape[2]
    X = delaunay.transform[simplex,:delaunay.ndim]
    Y = point - delaunay.transform[simplex,delaunay.ndim]
    b = X.dot(np.transpose(Y))
    bcoords = np.concatenate([b, [1 - b.sum()]])
    return bcoords

In [31]:
delaunay = Delaunay(vertices)

In [32]:
point = np.array([0,0,0])

In [33]:
delaunay.find_simplex(point)

array(0, dtype=int32)

In [34]:
barycentric_coords3_delaunay(delaunay, point)

array([0.        , 0.29166667, 0.41666667, 0.29166667])

In [35]:
barycentric_coords3(vertices, point)

array([0.29166667, 0.29166667, 0.41666667, 0.        ])

array([[3, 1, 2, 0]], dtype=int32)

In [20]:
delaunay.vertices

array([[3, 1, 2, 0]], dtype=int32)

In [24]:
delaunay.points

array([[-5.,  5.,  0.],
       [-5., -5.,  0.],
       [ 7.,  0.,  0.],
       [ 0.,  0.,  7.]])

In [23]:
def barycentric_coords2(vertices, point):
    v = vertices[1]-vertices[0]
    d = np.linalg.norm(v)
    u = v/d
    p = point - vertices[0]
    f = np.dot(p, u)/d
    return 1.0-f, f

In [17]:
all_bcoords3 = []

for ii in range(n_points):
    
    alpha, beta, gamma = barycentric_coords3(vertices, points[ii])
    all_bcoords3.append(np.array([alpha, beta, gamma]))

    if (alpha<0):
        alpha = 0.0
        beta, gamma = barycentric_coords2([vertices[1], vertices[2]], points[ii])
        if beta<0:
            beta = 0.0
            gamma = 1.0
        elif beta>1:
            beta = 1.0
            gamma = 0.0
    elif (beta<0):
        beta = 0.0
        alpha, gamma = barycentric_coords2([vertices[0], vertices[2]], points[ii])
        if alpha<0:
            alpha = 0.0
            gamma = 1.0
        elif alpha>1:
            alpha = 1.0
            gamma = 0.0
    elif (gamma<0):
        gamma = 0.0
        alpha, beta = barycentric_coords2([vertices[0], vertices[1]], points[ii])
        if alpha<0:
            alpha = 0.0
            beta = 1.0
        elif alpha>1:
            alpha = 1.0
            beta = 0.0

    v_points[ii] = alpha*v_vertices[0]+beta*v_vertices[1]+gamma*v_vertices[2]

In [18]:
plt.rcParams["figure.figsize"] = (6,6)

ax=plt.gca()

for ii in range(n_points):
    plt.arrow(points[ii,0], points[ii,1], v_points[ii,0], v_points[ii,1],
        head_width=0.2, head_length=0.4, fc='k', ec='k')
    
for ii in range(n_vertices):
    plt.arrow(vertices[ii,0], vertices[ii,1], v_vertices[ii,0], v_vertices[ii,1],
              head_width=0.2, head_length=0.4, fc='r', ec='r', lw=2.0)

    
plt.axis('scaled')
plt.xlim(-lbox, lbox)
plt.ylim(-lbox, lbox)

plt.show()